In [1]:
#training bigram tagger

from nltk.corpus import brown

brown_tagged_sents = brown.tagged_sents()
size = int(len(brown_tagged_sents) * 0.9)

train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]


In [4]:
import nltk
def build_backoff_tagger(train_sents):
    t0 = nltk.DefaultTagger('NN')
    t1 = nltk.UnigramTagger(train_sents, backoff=t0)
    t2 = nltk.BigramTagger(train_sents, backoff=t1)
    return t2

In [5]:
ngram_tagger = build_backoff_tagger(train_sents)

In [6]:
ngram_tagger.evaluate(test_sents)

0.9125751765470128

In [7]:
#loading train data
import pandas as pd
train_file_path = "project_2_train/project_2_train/dara 1_train.csv"
train_data = pd.read_csv(train_file_path)
train_data.head()

,example_id,text,aspect_term,term_location,class
0,2333_1,Obviously one of the most important features o...,human interface,69--84,0
1,1805_1,Good for every day computing and web browsing.,every day computing,9--28,1
2,2782_2,while the keyboard itself is alright[comma] th...,mouse command buttons,115--136,-1
3,1385_0,Again[comma] the same problem[comma] the right...,right speaker,29--42,-1
4,1423_0,My problem was with DELL Customer Service.,DELL Customer Service,20--41,-1


In [8]:
text_data = train_data[" text"]
new_text=[]
for idx,text_record in enumerate(text_data):
    current_text = text_data[idx].replace("[comma]", ",")
    #current_text = current_text[:int(locations[0])] + "<aspect>" + current_text[int(locations[1]):]
    new_text.append([current_text])

In [9]:
new_text_aspect=[]
for idx,text_record in enumerate(new_text):
    aspect_loc = train_data.loc[idx," term_location"]
    locations = aspect_loc.split("--")
    current_text = text_record[0]
    current_text = current_text[:int(locations[0])] + "<aspect>" + current_text[int(locations[1]):]
    new_text_aspect.append(current_text)

In [11]:
#extracting words in context 0f 5 from target term
import nltk
from nltk import word_tokenize
from nltk.util import ngrams

sent_list=[]

remove_list = [",",".",".)","..)",").","?","!"]
    
    
for idx,text_word in enumerate(new_text_aspect):
    locations = train_data.loc[idx," term_location"].split("--")
    sent = text_word
    sent_left = sent[:int(locations[0])]
    #token_left = nltk.word_tokenize(sent_left)
    #print(token_left)
    #bigram_left = nltk.bigrams(token_left)
    #bigram_left = *map(' '.join, bigrm)
    sent_right = sent[int(locations[0])+8:]
    #print(sent_right)
    
    #token_right = nltk.word_tokenize(sent_right)
    if sent_right not in remove_list:
        sent_surf = ' '.join(sent_left.split()[-5:])+' '+' '.join(sent_right.split()[:5])
    else:
        sent_surf = ' '.join(sent_left.split()[-5:])
    #print(sent_surf)
    sent_list.append([sent_surf])

In [15]:
import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer, WordPunctTokenizer
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures
features_uni=[]
#stop_list = stopwords.words('english') +['.',',',')','?','!','(','-',':','The','I','It','We',"'",'They','My','But','us']

#stops = set(stop_list)
for line in sent_list:
    tokenizer = WordPunctTokenizer()
    tokens = nltk.word_tokenize(line[0])
    #print(tokens)
    #stemmer = PorterStemmer()
    #bigram_finder = BigramCollocationFinder.from_words(tokens)
    #bigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 500)
    #bigrams = ngrams(line,2)
    #for bigram_tuple in bigrams:
    #    x = "%s %s" % bigram_tuple
    #    tokens.append(x)
    tagged_tokens = ngram_tagger.tag(tokens)
    #result = [b for b in nltk.bigrams(line) if b[0] not in stops and b[1] not in stops]
    #new_token = []
    #for token in tokens:
        #if token not in stops:
     #       new_token.append(token)
    features_uni.append(tagged_tokens)

In [46]:
#features_uni

In [19]:
new_tagged_Sent=[]
for feature in features_uni:
    sent = ""
    for tuples in feature:
        sent = sent +" " +tuples[0]+"_"+tuples[1]
    
    new_tagged_Sent.append(sent.strip())

In [23]:

text_df_tagged = pd.DataFrame({'text_tagged':new_tagged_Sent})

In [24]:
text_df_tagged.head()

,text_tagged
0,any_DTI computer_NN is_BEZ the_AT ``_``
1,Good_JJ for_IN and_CC web_NN browsing_VBG ._.
2,hollow_JJ sound_NN when_WRB using_VBG the_AT
3,"Again_RB ,_, the_AT same_AP problem_NN ,_, the..."
4,My_PP$ problem_NN was_BEDZ with_IN


In [47]:
#bag of words with pos tage
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_bag_tagged = count_vect.fit_transform(text_df_tagged["text_tagged"])
X_term_freq_tagged = pd.DataFrame(X_train_bag_tagged.toarray(),columns=count_vect.get_feature_names())

In [48]:
X_term_freq_tagged.head()

,0_nn,10,100_cd,1024_cd,10_cd,10th_nn,11_nn,13_cd,14_cd,15,...,yes_rb,yet_cc,yet_rb,you_nn,you_ppo,you_ppss,your_pp,yourself_ppl,yummy_nn,zero_cd
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_bag_tagged)
#X_train_tfidf.shape
X_tfidf_tagged = pd.DataFrame(X_train_tfidf.toarray())

In [44]:
#dropping columns with sum less than 2
cols_with_less_than_25 = [col for col in X_tfidf_tagged.columns 
                                 if abs(X_tfidf_tagged[col].sum()) < 1]
reduced_X_train_tagged = X_tfidf_tagged.drop(cols_with_less_than_25, axis=1)

In [45]:
reduced_X_train_tagged.head()

,1,2,4,7,15,19,20,22,25,30,...,2439,2440,2443,2446,2447,2448,2450,2451,2452,2455
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
